In [55]:
from selenium import webdriver
from selenium.webdriver.edge.options import Options
from icalendar import Calendar, Event, vCalAddress, vText
from datetime import datetime, date, timedelta
import os
import re
import configparser

In [3]:
config = configparser.ConfigParser()
config.read('config.ini')

['config.ini']

In [61]:
DRIVER_PATH = 'msedgedriver.exe'
option = Options()
option.headless = True
driver = webdriver.Edge(executable_path=DRIVER_PATH)
driver.set_window_size(1920, 1080)
driver.implicitly_wait(10)
driver.get('https://cas.ut-capitole.fr/cas/login?service=https%3A%2F%2Fade-production.ut-capitole.fr%2Fdirect%2Fmyplanning.jsp')

In [62]:
user = driver.find_element_by_id("username")
user.send_keys(config.get('default', 'username'))
pas = driver.find_element_by_id("password")
pas.send_keys(config.get('default', 'password'))
login = driver.find_element_by_id("connexion").click()

In [63]:
hours = {}
legend = driver.find_elements_by_class_name("slot")
for l in legend:
    heure = l.text[:-1] if len(l.text) > 5 else l.text
    top = l.get_attribute("style").split("; ")[-1]
    top_nb = int(re.findall(r'\d+', top)[0])+8
    hours[top_nb] = heure

In [64]:
month_french = {
    'January': 'janvier',
    'February': 'février',
    'March': 'mars',
    'April': 'avril',
    'May': 'mai',
    'June': 'juin',
    'July': 'juillet',
    'August': 'août',
    'September': 'septembre',
    'October': 'octobre',
    'November': 'novembre',
    'December': 'décembre'
}

In [65]:
cal = Calendar()
today = date.today()
current = today - timedelta(days=today.weekday())
current_week = today.isocalendar().week
current_top = 0

duration = {
    57: [1, 0],
    112: [2, 0],
    168: [3, 0],
    167: [3, 0]
}

for _ in range(4):
    planning = driver.find_element_by_id("Planning")
    planning_div = planning.find_elements_by_xpath("./*")
    planning_data = planning.find_elements_by_xpath("//div[@aria-owns='Planning']")

    for div, data in zip(planning_div, planning_data):
        heigh_tag = div.find_element_by_tag_name("table").get_attribute("style").split("; ")[-1]
        duration_tag = int(re.findall(r'\d+', heigh_tag)[0])
        real_duration = duration[duration_tag]

        top = div.get_attribute("style").split("; ")[-1]
        top_nb = int(re.findall(r'\d+', top)[0])
        list_data = [x.strip() for x in list(filter(lambda x: x.replace(" ", "") != "", data.get_attribute("aria-label").split('null')))]

        if current_top > top_nb:
            current = current + timedelta(days=1)
        current_top = top_nb

        year = current.year
        month = current.month
        day = current.day

        hour_class = hours[top_nb].split("h")
        event = Event()
        event.add('summary', list_data[0])
        event.add('dtstart', datetime(year, month, day, int(hour_class[0]), int(hour_class[1])))

        duration_mins = int(hour_class[1])+real_duration[1]
        duration_hours = int(hour_class[0])+real_duration[0]

        if duration_mins == 60:
            duration_hours+=1
            duration_mins=0

        event.add('dtend', datetime(year, month, day, duration_hours, duration_mins))

        if len(list_data) in [4, 5]:
            event['location'] = vText(list_data[1])
        else:
            event['location'] = vText('Inconnu')
        # Adding events to calendar
        cal.add_component(event)


    current_week+=1
    next_week = current + timedelta(days=3)
    next_day = next_week.strftime('%d')
    next_month = month_french[next_week.strftime('%B')]
    next_year = next_week.strftime('%y')
    next_week_str = f'({current_week}){next_day} {next_month} {next_year}'
    button_week = driver.find_element_by_xpath(f"//button[text()='{next_week_str}']")
    button_week.click()

StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: MicrosoftEdge=99.0.1150.46)


In [25]:
f = open(os.path.join("./", 'example.ics'), 'wb')
f.write(cal.to_ical())
f.close()